# PHY482 Group Project
Authors: **Kyle Taft and Thong Van**

# Abstract

Our code and computation will initially focus on a monochromatic plane wave interacting with different components essential to manipulating light via amplitude, phase, and polarization. We first explore the simple cases of the half and quarter wave plate. Next, we describe how our light wave changes with respect to an electro-optic modulator (EOM) or a voltage controlled birefringent crystal. We will compute consequences from such interactions such as sidebands and identify interesting scenarios at extremes of our set parameters(frequency, voltage, index of refraction, materials, etc).

We then decipher how our light wave changes with respect to a basic SLM unit cell in the following respective order: cover glass, birefringent crystal, mirror coating. We combine our basic SLM cell with other optical approaches such as a wave plate or even a Machl-Zehnder interferometer to design an architecture capable of creating bright, mild, and dark spots by interference. Finally, we compute how to create different kinds of patterns or even images by controlling contrast between brightness and maybe even colors. If time permits, we will compute the voltages and polarization of different incident light waves needed to create an image

# Code Setup

In [1]:
# Import packages
import numpy as np
import matplotlib.pyplot as plt
import imageio
import os
import shutil
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

In [2]:
# Define constants
# c = 299792458  # Speed of light (m/s)
c = 1 
k = 4  # Wavenumber (rad/m)
w = k * c  # Angular frequency (rad/s)
E0 = 1  # Amplitude of electric field (V/m)
B0 = E0 / c  # Amplitude of magnetic field (T)

# Define the time parameters
tmax = 10
delta_t = 0.1

# Choose a propagation direction 
prop_dir = np.array([1, 0, 0])

# Choose a polarization direction
pol_dir = np.array([0, 1, 0])

In [3]:
# Define fundamental functions
def electric_field(x, y, z, t=0, prop_dir=np.array([1, 0, 0]), pol_dir=np.array([0, 1, 0])):
    """
    Calculate the electric field at a given point in space and time
    :param x,y,z: spacial coordinate of the point
    :param t: time
    :param prop_dir: propagation direction of the wave
    :param pol_dir: polarization direction of the electric field
    :param index: index specifying the direction
    :return: Ex, Ey, Ez: electric field components at the given point and time
    """
    # Define the electric field equation
    E = E0 * np.exp(1j * (k * (prop_dir[0] * x + prop_dir[1] * y + prop_dir[2] * z) - w * t))

    # Calculate electric field components
    Ex = E.real * pol_dir[0]
    Ey = (E * pol_dir[1]).real
    Ez = (E * pol_dir[2]).real
    return Ex, Ey, Ez

def magnetic_field(x, y, z, t=0, prop_dir=np.array([1, 0, 0]), pol_dir=np.array([0, 1, 0])):
    """
    Calculate the magnetic field at a given point in space and time
    :param x,y,z: spacial coordinate of the point
    :param t: time
    :param prop_dir: propagation direction of the wave
    :param pol_dir: polarization direction of the electric field
    :param index: index specifying the direction
    :return: Bx, By, Bz: magnetic field components at the given point and time
    """
    # Find the direction of the magnetic field
    B_direction = np.cross(prop_dir, pol_dir)

    # Calculate the magnetic field
    B = B0 * np.exp(1j * (k * (prop_dir[0] * x + prop_dir[1] * y + prop_dir[2] * z) - w * t))

    # Calculate magnetic field components
    Bx = B.real * B_direction[0]
    By = B.real * B_direction[1]
    Bz = B.real * B_direction[2]
    return Bx, By, Bz

def max_coordinate(prop_dir, t, index):
    """
    Define how far the wave has propagated in the given direction
    :param prop_dir: propagation direction of the wave
    :param t: time
    :param index: index specifying the direction
    :return: maximum coordinate in the given direction
    """
    if prop_dir[index] == 0:
        return 1
    else:
        return prop_dir[index] * c * t


# Monochromatic wave

In [4]:
# Mono-chromatic wave

def simple_wave(save_as):
	for i, t in enumerate(np.arange(0, tmax, delta_t)):
		fig = plt.figure()
		ax = fig.add_subplot(111, projection='3d')
		
		# Define grid for plotting
		num_points = 100
		x = np.linspace(0, max_coordinate(prop_dir, tmax, 0), num_points)
		y = np.linspace(0, max_coordinate(prop_dir, tmax, 1), num_points)
		z = np.linspace(0, max_coordinate(prop_dir, tmax, 2), num_points)

		# Calculate the field components
		Ex, Ey, Ez = electric_field(x, y, z, t=t, prop_dir=prop_dir, pol_dir=pol_dir)
		Bx, By, Bz = magnetic_field(x, y, z, t=t, prop_dir=prop_dir, pol_dir=pol_dir)

		# Plot in 3D
		ax.plot(x, Ey, Ez, color='b', label='Electric field')
		ax.plot(x, By, Bz, color='r', label='Magnetic field')

		# Show the propagation direction
		ax.plot([0, prop_dir[0] * max_coordinate(prop_dir, tmax, 0)], 
				[0, prop_dir[1] * max_coordinate(prop_dir, tmax, 1)], 
				[0, prop_dir[2] * max_coordinate(prop_dir, tmax, 2)], color='k', linestyle='--', label='Propagation axis')

		ax.set_xlim(-1, max_coordinate(prop_dir, 10, 0))
		ax.set_ylim(-1, max_coordinate(prop_dir, 10, 1))
		ax.set_zlim(-1, max_coordinate(prop_dir, 10, 2))
		ax.set_box_aspect([3, 1, 1])
		# Set the labels and title
		ax.set_xlabel('X')
		ax.set_ylabel('Y')
		ax.set_zlabel('Z')
		ax.set_title('Mono-chromatic plane wave in 3D')
		ax.legend()
		ax.figure.savefig(f'figures/{save_as}_{i:02d}.png')
		plt.close()


# Half-wave plate and Quarter-wave plate

A waveplate is characterized by the difference in phase it introduces to an incident wave given by $ \Gamma = \frac{2 \pi \Delta n L}{\lambda _0}$. Here, $\Delta n$ is the difference of refractive index between the fast and slow axis of the birefringent crystal, $L$ is the crystals thickness, and $\lambda _0$ is the wavelength of the incident light in a vacuum. For the following demonstration, we place a waveplate at the $x=5$ plane represented by the green surface. Furthermore, we let the propagation of our light be normal to our waveplate and only show the electric field so that it is easier to see the change in polarization.

In [5]:
# Waveplate

def waveplate(deltan, lambda0, L, platetype, save_as, plate=5):
    """
    Generate a representation of a wave interacting with a waveplate
    :param deltan: difference of refractive index between fast and slow axis
    :param lambda0: wavelength of the light wave in a vacuum
    :param L: Length of the waveplate
    :param save_as: Base name of saved images
    :param plate: location of the plate along the x-axis
    """
    def gamma(deltan, lambda0, L):
        """
        Determine gamma (phase shift) as shown in the equation above
        :param deltan: difference of refractive index between fast and slow axis
        :param lambda0: wavelength of the light wave in a vacuum
        :param L: Length of the waveplate
        """
        return 2*np.pi*deltan*L / lambda0
    
    phase = gamma(deltan, lambda0, L)

    # Decide whether to include a half or quarter waveplate
    if platetype == 'half':
        pol_dir2 = np.array([0,np.cos(phase),np.sin(phase)])
    elif platetype == 'quarter':
        pol_dir2 = np.array([0, 1 , np.exp(1j*phase)])
        
    for i, t in enumerate(np.arange(0, tmax, delta_t)):
        fig = plt.figure()
        ax = fig.add_subplot(111, projection='3d')
        
        # Define grid for plotting
        num_points = 50
        
        x = np.linspace(0, plate, num_points)
        y = np.linspace(0, max_coordinate(prop_dir, tmax, 1), num_points)
        z = np.linspace(0, max_coordinate(prop_dir, tmax, 2), num_points)

        
        x2 = np.linspace(plate, max_coordinate(prop_dir, tmax, 0), num_points)
        y2 = np.linspace(plate, max_coordinate(prop_dir, tmax, 1), num_points)
        z2 = np.linspace(plate, max_coordinate(prop_dir, tmax, 2), num_points)

        # Calculate the field components
        Ex, Ey, Ez = electric_field(x, y, z, t=t, prop_dir=prop_dir, pol_dir=pol_dir)
        # Calculate the field components after waveplate
        Ex2, Ey2, Ez2 = electric_field(x, y, z, t=t, prop_dir=prop_dir, pol_dir=pol_dir2)

        
        # Plot in 3D
        ax.plot(x, Ey, Ez, color='b', label='Electric field')
    
        # Show the propagation direction
        ax.plot([0, prop_dir[0] * max_coordinate(prop_dir, tmax, 0)], 
                [0, prop_dir[1] * max_coordinate(prop_dir, tmax, 1)], 
                [0, prop_dir[2] * max_coordinate(prop_dir, tmax, 2)], color='k', linestyle='--', label='Propagation axis')

        ax.plot(x2, Ey2.real, Ez2.real, color='b')
    
        ax.set_xlim(0, max_coordinate(prop_dir, 10, 0))
        ax.set_ylim(-1, max_coordinate(prop_dir, 10, 1))
        ax.set_zlim(-1, max_coordinate(prop_dir, 10, 2))

        x_start = plate
        y_limits = [-1, max_coordinate(prop_dir, 10, 1)]
        z_limits = [-1, max_coordinate(prop_dir, 10, 2)]
        vertices = [(x_start, y_limits[0], z_limits[0]),
            (x_start, y_limits[0], z_limits[1]),
            (x_start, y_limits[1], z_limits[1]),
            (x_start, y_limits[1], z_limits[0])]
        verts = [vertices]

        ax.view_init(15, 45)
        
        ax.add_collection3d(Poly3DCollection(verts, alpha=0.3, facecolor='g'))
        ax.set_box_aspect([3, 1, 1])
        # Set the labels and title
        ax.set_xlabel('X')
        ax.set_ylabel('Y')
        ax.set_zlabel('Z')
        ax.set_title('Mono-chromatic plane wave in 3D')
        ax.legend()
        #plt.show()
        ax.figure.savefig(f'figures/{save_as}_{i:02d}.png')
        plt.close()


In [6]:
## Save the collection of images produced from the function.
deltan = .5*(1/6) # Difference between refractive indices of the wave plate
L = 5e-7 # Length of the waveplate itself [m]
lambda0 = 5e-7 # Wavelength of the light in vacuum [m]
plate_type = 'half' # Choose 'quarter' or 'half' for the type of waveplate

# Clear output directory
if os.path.exists('figures'):
    shutil.rmtree('figures')
os.makedirs('figures')
     

image_name = f'{plate_type}_phase_30' # Base name for images
waveplate(deltan, L, lambda0, plate_type, image_name) # Produce set of images for wave

# image_name = 'simple_wave'
# simple_wave(image_name) # Produce set of images for wave


### Create a GIF from the images

# Get list of input PNG files
png_files = sorted([os.path.join('figures', file) for file in os.listdir('figures') if file.endswith('.png')])

# Read PNG images and create GIF
with imageio.get_writer(image_name+'.gif', mode='I', duration=0.2) as writer:
    for png_file in png_files:
        image = imageio.imread(png_file)
        writer.append_data(image)


C:\Users\2003k\AppData\Local\Temp\ipykernel_14176\3806205615.py:28: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread(png_file)
